## Homework

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-11 16:12:36--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 172.253.115.138, 172.253.115.100, 172.253.115.101, ...
Connecting to docs.google.com (docs.google.com)|172.253.115.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/25cs55cdgcbf9ou4sum7tn6dq40srvgo/1652285550000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-11 16:12:38--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/25cs55cdgcbf9ou4sum7tn6dq40srvgo/1652285550000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 172.253.122.132, 2607:f8b0:4004:c09::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [3]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [5]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [6]:
df.shape

(20000, 292)

In [7]:
df.head(20)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
5,5,2013-04-30,60,33.0,12.0,NaN,NaN,NaN,NaN,NaN,...,11,6,0,14,22,1,1,65,7,10600000
6,6,2014-01-22,59,43.0,4.0,9.0,1.0,1971.0,3.0,5.0,...,13,5,0,13,29,1,2,76,10,8300000
7,7,2012-01-26,39,23.0,11.0,NaN,NaN,NaN,NaN,NaN,...,18,13,1,7,19,1,2,51,1,990000
8,8,2015-04-24,59,44.0,3.0,9.0,1.0,1967.0,3.0,5.0,...,6,2,0,9,28,0,7,66,13,8600000
9,9,2014-05-16,60,33.0,3.0,17.0,1.0,1983.0,2.0,10.0,...,18,5,0,12,26,1,3,82,7,10850000


In [8]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [9]:
df.isna().any().sum()

51

Так как объектов с пропусками немного, можно их удалить из выборки

In [10]:
df= df.dropna(how='any')

In [11]:
df2 = df.drop(drop_columns, axis=1)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [12]:
df_sc_base = pd.concat([df2[num_columns], pd.get_dummies(df2[cat_columns])], axis=1)
df_sc_base.shape

(3959, 385)

In [13]:
X = df_sc_base.drop(['price'], axis=1)
y = df_sc_base['price']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [15]:
model_base = DecisionTreeRegressor(max_depth=5)
model_base.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5)

Проверьте качество на отложенной выборке.

In [16]:
y_pred = model_base.predict(X_test)
mean_squared_error(y_test, y_pred, squared = False)

4364503.946791289

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [17]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [18]:
num_columns1 = np.concatenate((num_columns, ["month_year_cnt", "week_year_cnt", '%floor', '%sq_kitch']))

Добавьте следующие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [19]:
df['datetime'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
df.insert(1, 'month', df['datetime'].dt.month)
df.insert(1, 'week_day', df['datetime'].dt.weekday)
df = df.drop(drop_columns, axis=1)
df = df.drop(['datetime'], axis=1)

In [20]:
cat_columns1 = np.concatenate((cat_columns, ['month', 'week_day']))

In [21]:
df['%floor'] = df['floor']/df['max_floor']
df['%sq_kitch'] = df['kitch_sq']/df['full_sq']

In [22]:
df

,week_day,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt,%floor,%sq_kitch
4,2,10,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,8,0,1,34,5,7900000,195,56,1.000000,0.100000
6,2,1,59,43.0,4.0,9.0,1.0,1971.0,3.0,5.0,...,29,1,2,76,10,8300000,166,55,0.444444,0.084746
12,4,1,39,21.0,3.0,14.0,1.0,1992.0,1.0,8.0,...,9,0,1,10,2,3000000,166,68,0.214286,0.205128
16,0,3,106,87.0,24.0,30.0,4.0,2004.0,1.0,15.0,...,51,1,7,112,11,28500000,228,48,0.800000,0.141509
20,2,12,38,18.0,4.0,17.0,1.0,1985.0,1.0,8.0,...,28,2,6,84,3,8000000,325,33,0.235294,0.210526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19975,4,9,32,18.0,5.0,9.0,1.0,1978.0,1.0,6.0,...,23,0,1,29,7,6150000,228,44,0.555556,0.187500
19978,2,1,59,36.0,4.0,5.0,2.0,1960.0,2.0,7.0,...,63,1,13,119,13,10000000,166,68,0.800000,0.118644
19984,3,5,62,60.0,5.0,9.0,1.0,1976.0,3.0,8.0,...,24,0,5,57,14,10300000,231,54,0.555556,0.129032
19995,0,9,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,29,0,1,46,10,7200000,228,58,0.222222,0.111111


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [23]:
df3 = pd.concat([df[num_columns], pd.get_dummies(df[cat_columns])], axis=1)

In [24]:
X_tr, X_tt, y_tr, y_tt = train_test_split(df3.drop(['price'], axis=1), df3['price'], test_size=0.2)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [34]:
model_1 = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid={'max_depth': range(2, 21, 2), 'max_features': np.arange(0.1, 0.8, 0.1), 'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']}, scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 5, refit=True)
model_1.fit(X_tr, y_tr)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': range(2, 21, 2),
                         'max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7])},
             scoring='neg_root_mean_squared_error')

***Первый результат:***

In [67]:
mean_squared_error(y_tt, model_1.predict(X_tt), squared = False)

4257347.256186652

In [63]:
model_2 = GridSearchCV(estimator = RandomForestRegressor(), param_grid = {'n_estimators': np.arange(50, 301, 100), 'max_depth': np.arange(5, 20, 6), 'max_features': np.arange(0.1, 0.8, 0.2),'random_state': [42]}, refit = True, scoring = 'neg_root_mean_squared_error', n_jobs =-1, cv = 3)
model_2.fit(X_tr, y_tr)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': array([ 5, 11, 17]),
                         'max_features': array([0.1, 0.3, 0.5, 0.7]),
                         'n_estimators': array([ 50, 150, 250]),
                         'random_state': [42]},
             scoring='neg_root_mean_squared_error')

***Второй:***

In [68]:
mean_squared_error(y_tt, model_2.predict(X_tt), squared = False)

3739513.5253307754

In [64]:
model_3 = CatBoostRegressor(loss_function = 'RMSE')
results = model_3.grid_search({'n_estimators': np.arange(100, 301, 80), 'max_depth' : np.arange(5, 22, 7)}, X = X_tr, y=y_tr, cv = 3)

0:	learn: 10332472.3254296	test: 10062910.7412775	best: 10062910.7412775 (0)	total: 78.5ms	remaining: 7.78s
1:	learn: 10095940.4672197	test: 9828683.7472942	best: 9828683.7472942 (1)	total: 106ms	remaining: 5.17s
2:	learn: 9877262.5242297	test: 9614410.6637131	best: 9614410.6637131 (2)	total: 133ms	remaining: 4.29s
3:	learn: 9652709.9860891	test: 9394385.3371294	best: 9394385.3371294 (3)	total: 158ms	remaining: 3.8s
4:	learn: 9454119.7651169	test: 9201296.7943871	best: 9201296.7943871 (4)	total: 183ms	remaining: 3.48s
5:	learn: 9246383.3284471	test: 8997083.2973681	best: 8997083.2973681 (5)	total: 208ms	remaining: 3.26s
6:	learn: 9041953.4010606	test: 8792462.6771525	best: 8792462.6771525 (6)	total: 235ms	remaining: 3.12s
7:	learn: 8856447.4735206	test: 8609151.4124010	best: 8609151.4124010 (7)	total: 260ms	remaining: 2.99s
8:	learn: 8688277.7433022	test: 8438679.1937105	best: 8438679.1937105 (8)	total: 289ms	remaining: 2.93s
9:	learn: 8505662.4429575	test: 8255758.1297469	best: 825575

***Третий:***

In [72]:
mean_squared_error(y_tt, model_3.predict(X_tt), squared=False)

3554009.885724668

Вне всяких сомнений:

In [73]:
best_model_1 = model_3
best_model_2 = model_2

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [77]:
X_tmp = pd.concat([X_tr, y_tr], axis = 1)
invest_train_X = X_tmp[X_tmp['product_type_Investment'] == 1].drop(['product_type_OwnerOccupier'], axis = 1)
invest_train_Y = invest_train_X['price']
invest_train_X = invest_train_X.drop(['price'], axis = 1)
owner_train_X = X_tmp[X_tmp['product_type_Investment'] == 0].drop(['product_type_OwnerOccupier'], axis = 1)
owner_train_Y = owner_train_X['price']
owner_train_X = owner_train_X.drop(['price'], axis = 1)
model_invest = best_model_1
model_invest.fit(invest_train_X, invest_train_Y)
model_owner = best_model_2
model_owner.fit(owner_train_X, owner_train_Y)
X_tt_ = X_tt.drop(['product_type_OwnerOccupier'], axis=1)

Learning rate set to 0.145602
0:	learn: 5438069.6121320	total: 32.1ms	remaining: 8.32s
1:	learn: 5110996.1319225	total: 57.6ms	remaining: 7.43s
2:	learn: 4824124.8331363	total: 83.6ms	remaining: 7.16s
3:	learn: 4616284.5309171	total: 110ms	remaining: 7.04s
4:	learn: 4419183.9445302	total: 135ms	remaining: 6.86s
5:	learn: 4257786.6202817	total: 159ms	remaining: 6.75s
6:	learn: 4104251.7654879	total: 191ms	remaining: 6.89s
7:	learn: 3982027.5545703	total: 221ms	remaining: 6.96s
8:	learn: 3898814.4710075	total: 246ms	remaining: 6.86s
9:	learn: 3807113.2987686	total: 271ms	remaining: 6.78s
10:	learn: 3724079.4023265	total: 295ms	remaining: 6.67s
11:	learn: 3651987.9018432	total: 320ms	remaining: 6.61s
12:	learn: 3596560.3350671	total: 344ms	remaining: 6.54s
13:	learn: 3531179.4507557	total: 371ms	remaining: 6.52s
14:	learn: 3482507.0146104	total: 396ms	remaining: 6.47s
15:	learn: 3441149.0930681	total: 422ms	remaining: 6.43s
16:	learn: 3398626.1199991	total: 448ms	remaining: 6.41s
17:	lear

In [80]:
coef_1 = df3[df3['product_type_Investment'] == 1].shape[0] / df3.shape[0]
coef_2 = 1 - coef_1
y_pred_tot = model_invest.predict(X_tt_) * coef_1 + model_owner.predict(X_tt_) * coef_2
mean_squared_error(y_tt, y_pred_tot, squared = False)

3778776.885970398

Возможно, я делаю что-то не так, но результат у catboost лучше

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).